In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from shapely import wkt
import pyogrio
from tqdm import tqdm

geopackage_path = "C:/Users/eleonore.kong/Documents/InSAR/DATA/lpf_egms_data.gpkg"
n = 19829367
size = 200000
chunks = round(n/size) + 1
skip=0

In [2]:
df = pyogrio.read_dataframe(geopackage_path, layer='data_rga', max_features=100000)

In [5]:
winter = df[df['season']=='winter'].groupby(['pid', 'value_type']).max('value_diff_abs_denorm').reset_index()
summer = df[df['season']=='summer'].groupby(['pid', 'value_type']).min('value_diff_abs_denorm').reset_index()
data = pd.concat([winter, summer], axis=0)
# data.drop(['value_diff_abs'], axis=1, inplace=True)
# data = data.merge(df[['geometry', 'Type', 'Burst ID', 'season','Date']], on='pid', how='left')
# data.sort_values(by=['pid', 'Date'], inplace=True)
# data['value_diff'] = data['Value'].diff()
# data.loc[data['time_gap']<1, 'value_diff'] = np.nan
# data.reset_index(inplace=True, drop=True)
data

,pid,value_type,Value,year,month,time_gap,value_diff,is_rga,value_diff_abs,month_trough,...,gap_insar_swi_trough,gap_insar_swi_peak,is_swi_close,NIVEAU,value_diff_abs_denorm,median_value_diff_abs_2016_2021,median_value_diff_abs_denorm_2016_2021,gap_insar_swi_trough_days,gap_insar_swi_peak_days,amp_diff_to_median
0,10S39QasDg,winter_2016,0.715249,2016,3,NaN,NaN,True,NaN,9,...,NaN,-9.0,True,2.0,NaN,9.76,8.281150,NaN,-253.0,NaN
1,10S39QasDg,winter_2017,0.716117,2017,2,6.2,13.625,True,13.625,8,...,NaN,1.0,True,2.0,11.560520,9.76,8.281150,NaN,27.0,3.279370
2,10S39QasDg,winter_2018,0.716851,2018,3,6.4,9.760,True,9.760,9,...,NaN,0.0,True,2.0,8.281150,9.76,8.281150,NaN,0.0,0.000000
3,10S39QasDg,winter_2019,0.719545,2019,2,5.8,11.561,True,11.561,8,...,NaN,-10.0,True,2.0,9.809260,9.76,8.281150,NaN,-298.0,1.528110
4,10S39QasDg,winter_2020,0.718671,2020,2,6.2,9.406,True,9.406,9,...,NaN,1.0,True,2.0,7.980789,9.76,8.281150,NaN,0.0,-0.300361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8339267,40cEO62jrK,summer_2017,0.705187,2017,7,5.6,-5.445,True,5.445,8,...,-1.0,NaN,True,0.0,4.619966,5.07,4.301786,-28.0,NaN,0.318179
8339268,40cEO62jrK,summer_2018,0.706779,2018,8,6.6,-4.550,True,4.550,8,...,0.0,NaN,True,0.0,3.860577,5.07,4.301786,0.0,NaN,-0.441209
8339269,40cEO62jrK,summer_2019,0.706941,2019,7,6.0,-5.070,True,5.070,9,...,-2.0,NaN,True,0.0,4.301786,5.07,4.301786,-51.0,NaN,0.000000
8339270,40cEO62jrK,summer_2020,0.704854,2020,6,6.0,-6.510,True,6.510,8,...,-2.0,NaN,True,0.0,5.523595,5.07,4.301786,-38.0,NaN,1.221809


In [7]:
df_pivot = data.pivot_table(index='pid', columns=['value_type'], values='value_diff_abs_denorm', aggfunc='max')
# df_pivot.drop('winter_2022', axis=1, inplace=True)
df_pivot

value_type,summer_2016,summer_2017,summer_2018,summer_2019,summer_2020,summer_2021,winter_2016,winter_2017,winter_2018,winter_2019,winter_2020,winter_2021
pid,,,,,,,,,,,,
10S39QasDg,10.824041,7.658367,7.523459,8.722359,9.960289,10.381983,NaN,11.560520,8.281150,9.809260,7.980789,8.109758
10S39QasDr,3.882638,7.957880,10.320893,11.056524,7.660064,2.745677,NaN,8.589997,7.950244,10.068895,13.241356,6.360195
10S39QasDs,2.935736,1.276960,4.992448,3.925910,0.509936,2.183984,NaN,1.890410,3.777426,5.732321,3.144461,0.005091
10S39QasDt,1.004599,2.533557,4.491845,3.743487,4.635238,NaN,NaN,4.251726,3.798638,NaN,3.318400,2.923009
10S39QasER,NaN,6.095470,3.469429,4.601299,6.150621,4.697177,NaN,4.329786,4.500330,4.827843,7.317279,3.555125
...,...,...,...,...,...,...,...,...,...,...,...,...
40cEJPNUfH,NaN,5.758624,5.135841,5.425172,5.829896,4.026879,NaN,6.136197,6.266862,4.559724,5.472687,4.071848
40cEO62jrG,NaN,2.929796,3.175855,3.354036,1.793684,1.503504,NaN,3.355733,3.352339,3.050280,3.031614,0.195150
40cEO62jrI,1.881077,2.316346,3.374399,5.342869,6.686859,4.754874,NaN,3.447368,4.796449,3.513550,5.447232,3.737548


In [8]:
# for col in df_pivot.columns:
#     if 'winter' in col:
#         df_pivot.loc[df_pivot[col] < 0, col] = 0
#     if 'summer' in col:
#         df_pivot.loc[df_pivot[col] > 0, col] = 0
#         df_pivot[col] = abs(df_pivot[col])
# df_pivot.reset_index(inplace=True)

gdf = df_pivot.merge(df[['pid','geometry']], on='pid', how='left').drop_duplicates()
gdf = gpd.GeoDataFrame(gdf, geometry='geometry', crs=2154)

In [9]:
gdf

,pid,summer_2016,summer_2017,summer_2018,summer_2019,summer_2020,summer_2021,winter_2016,winter_2017,winter_2018,winter_2019,winter_2020,winter_2021,geometry
0,10S39QasDg,10.824041,7.658367,7.523459,8.722359,9.960289,10.381983,NaN,11.560520,8.281150,9.809260,7.980789,8.109758,POINT (1046178.932 6306701.725)
12,10S39QasDr,3.882638,7.957880,10.320893,11.056524,7.660064,2.745677,NaN,8.589997,7.950244,10.068895,13.241356,6.360195,POINT (1047272.145 6306801.060)
25,10S39QasDs,2.935736,1.276960,4.992448,3.925910,0.509936,2.183984,NaN,1.890410,3.777426,5.732321,3.144461,0.005091,POINT (1047371.528 6306810.089)
39,10S39QasDt,1.004599,2.533557,4.491845,3.743487,4.635238,NaN,NaN,4.251726,3.798638,NaN,3.318400,2.923009,POINT (1047470.911 6306819.119)
51,10S39QasER,NaN,6.095470,3.469429,4.601299,6.150621,4.697177,NaN,4.329786,4.500330,4.827843,7.317279,3.555125,POINT (1050849.925 6307126.079)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19829307,40cEJPNUfH,NaN,5.758624,5.135841,5.425172,5.829896,4.026879,NaN,6.136197,6.266862,4.559724,5.472687,4.071848,POINT (668330.575 7110000.654)
19829318,40cEO62jrG,NaN,2.929796,3.175855,3.354036,1.793684,1.503504,NaN,3.355733,3.352339,3.050280,3.031614,0.195150,POINT (667821.733 7110052.186)
19829331,40cEO62jrI,1.881077,2.316346,3.374399,5.342869,6.686859,4.754874,NaN,3.447368,4.796449,3.513550,5.447232,3.737548,POINT (668021.436 7110071.449)
19829344,40cEO62jrJ,NaN,6.467952,3.370157,4.620814,2.973069,2.276468,NaN,4.372210,5.211355,4.054879,3.544943,2.977311,POINT (668121.287 7110081.080)


In [46]:
pyogrio.write_dataframe(gdf, geopackage_path, layer="data_processed", driver="GPKG")